# School timetabling with Timefold in a Python Notebook

This Python Jupyter Notebook solves a school timetabling problem with [Timefold](https://timefold.ai), the open source planning solver AI.
It optimizes the school schedule for students and teachers.

![School timetabling input output](https://timefold.ai/docs/timefold-solver/latest/_images/quickstart/school-timetabling/schoolTimetablingInputOutput.png)

This Notebook automatically assigns each lesson to a timeslot and a room, while adhering to hard and soft constraints: 

* A room can have at most one lesson at the same time.
* A teacher can teach at most one lesson at the same time.
* A student can attend at most one lesson at the same time.
* A teacher prefers to teach all lessons in the same room.
* A teacher prefers to teach sequential lessons and dislikes gaps between lessons.
* A student dislikes sequential lessons on the same subject.

## Dependencies

Add the Timefold solver dependency:

In [1]:
pip install timefold==1.16.0b0

  Obtaining dependency information for timefold==1.16.0b0 from https://files.pythonhosted.org/packages/fa/c4/07aff59a58cc72b3fbf665096086d55343ed655b1fbba29e4f8391da5467/timefold-1.16.0b0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 57.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: timefold
    Found existing installation: timefold 1.11.0b0
    Uninstalling timefold-1.11.0b0:
      Successfully uninstalled timefold-1.11.0b0

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



## Domain

Create the domain data classes:

### Room

The `Room` class represents a location where lessons are taught, for example, `Room A` or `Room B`. For simplicity’s sake, all rooms are without capacity limits and they can accommodate all lessons.

In [2]:
from dataclasses import dataclass

@dataclass
class Room:
    name: str

    def __str__(self):
        return f'{self.name}'

### Timeslot

The `Timeslot` class represents a time interval when lessons are taught, for example, `Monday 10:30 - 11:30` or `Tuesday 13:30 - 14:30`. For simplicity’s sake, all time slots have the same duration and there are no time slots during lunch or other breaks.

In [3]:
from dataclasses import dataclass
from datetime import time

@dataclass
class Timeslot:
    day_of_week: str
    start_time: time
    end_time: time

    def __str__(self):
        return f'{self.day_of_week} {self.start_time.strftime("%H:%M")}'


### Lesson

During a lesson, represented by the `Lesson` class, a teacher teaches a subject to a group of students, for example, `Math` by `A.Turing` for `9th grade` or `Chemistry` by `M.Curie` for `10th grade`. If a subject is taught multiple times per week by the same teacher to the same student group, there are multiple `Lesson` instances that are only distinguishable by `id`. For example, the 9th grade has six math lessons a week.

During solving, Timefold Solver changes the `timeslot` and `room` fields of the `Lesson` class, to assign each lesson to a time slot and a room. Therefore, those fields are annnoted with `@PlanningVariable` and the class with `@PlanningEntity`:

In [4]:
from timefold.solver.domain import (planning_entity, PlanningVariable, PlanningId)
from dataclasses import dataclass, field
from typing import Annotated

@planning_entity
@dataclass
class Lesson:
    id: Annotated[str, PlanningId]
    subject: str
    teacher: str
    student_group: str
    timeslot: Annotated[Timeslot | None, PlanningVariable] = field(default=None)
    room: Annotated[Room | None, PlanningVariable] = field(default=None)

## Constraints

The solver takes into account hard and soft constraints:

In [5]:
from timefold.solver.score import (constraint_provider, HardSoftScore, Joiners,
                                   ConstraintFactory, Constraint)
from datetime import time

@constraint_provider
def define_constraints(constraint_factory: ConstraintFactory):
    return [
        # Hard constraints
        room_conflict(constraint_factory),
        teacher_conflict(constraint_factory),
        student_group_conflict(constraint_factory),

        # Soft constraints
        teacher_room_stability(constraint_factory),
        teacher_time_efficiency(constraint_factory),
        student_group_subject_variety(constraint_factory),
    ]


def room_conflict(constraint_factory: ConstraintFactory) -> Constraint:
    # A room can accommodate at most one lesson at the same time.
    return (constraint_factory
            # Select each pair of 2 different lessons ...
            .for_each_unique_pair(Lesson,
                                  # ... in the same timeslot ...
                                  Joiners.equal(lambda lesson: lesson.timeslot),
                                  # ... in the same room ...
                                  Joiners.equal(lambda lesson: lesson.room))
            # ... and penalize each pair with a hard weight.
            .penalize(HardSoftScore.ONE_HARD)
            .as_constraint("Room conflict"))


def teacher_conflict(constraint_factory: ConstraintFactory) -> Constraint:
    # A teacher can teach at most one lesson at the same time.
    return (constraint_factory
            .for_each_unique_pair(Lesson,
                                  Joiners.equal(lambda lesson: lesson.timeslot),
                                  Joiners.equal(lambda lesson: lesson.teacher))
            .penalize(HardSoftScore.ONE_HARD)
            .as_constraint("Teacher conflict"))


def student_group_conflict(constraint_factory: ConstraintFactory) -> Constraint:
    # A student can attend at most one lesson at the same time.
    return (constraint_factory
            .for_each_unique_pair(Lesson,
                                  Joiners.equal(lambda lesson: lesson.timeslot),
                                  Joiners.equal(lambda lesson: lesson.student_group))
            .penalize(HardSoftScore.ONE_HARD)
            .as_constraint("Student group conflict"))


def teacher_room_stability(constraint_factory: ConstraintFactory) -> Constraint:
    # A teacher prefers to teach in a single room.
    return (constraint_factory
            .for_each_unique_pair(Lesson,
                                  Joiners.equal(lambda lesson: lesson.teacher))
            .filter(lambda lesson1, lesson2: lesson1.room != lesson2.room)
            .penalize(HardSoftScore.ONE_SOFT)
            .as_constraint("Teacher room stability"))


def to_minutes(moment: time) -> int:
    return moment.hour * 60 + moment.minute


def is_between(lesson1: Lesson, lesson2: Lesson) -> bool:
    difference = to_minutes(lesson1.timeslot.end_time) - to_minutes(lesson2.timeslot.start_time)
    return 0 <= difference <= 30


def teacher_time_efficiency(constraint_factory: ConstraintFactory) -> Constraint:
    # A teacher prefers to teach sequential lessons and dislikes gaps between lessons.
    return (constraint_factory.for_each(Lesson)
            .join(Lesson,
                  Joiners.equal(lambda lesson: lesson.teacher),
                  Joiners.equal(lambda lesson: lesson.timeslot.day_of_week))
            .filter(is_between)
            .reward(HardSoftScore.ONE_SOFT)
            .as_constraint("Teacher time efficiency"))


def student_group_subject_variety(constraint_factory: ConstraintFactory) -> Constraint:
    # A student group dislikes sequential lessons on the same subject.
    return (((constraint_factory.for_each(Lesson)
              .join(Lesson,
                    Joiners.equal(lambda lesson: lesson.subject),
                    Joiners.equal(lambda lesson: lesson.student_group),
                    Joiners.equal(lambda lesson: lesson.timeslot.day_of_week))
              .filter(is_between))
             .penalize(HardSoftScore.ONE_SOFT))
            .as_constraint("Student group subject variety"))


### TimeTable

The `TimeTable` class represents a single dataset. It wraps all `Timeslot`, `Room`, and `Lesson` instances. Furthermore, because it contains all lessons, each with a specific planning variable state, it is a planning solution, and it has a `score` field:

In [6]:
from timefold.solver.domain import (planning_solution, 
                                    PlanningEntityCollectionProperty, ProblemFactCollectionProperty,
                                    ValueRangeProvider,
                                    PlanningScore)
from timefold.solver.score import HardSoftScore
from dataclasses import dataclass, field
from typing import Annotated

@planning_solution
@dataclass
class Timetable:
    timeslots: Annotated[list[Timeslot], ProblemFactCollectionProperty, ValueRangeProvider]
    rooms: Annotated[list[Room], ProblemFactCollectionProperty, ValueRangeProvider]
    lessons: Annotated[list[Lesson],  PlanningEntityCollectionProperty]
    score: Annotated[HardSoftScore, PlanningScore] = field(default=None)

## Data generator

Generate some data for a small school timetable:

In [7]:
def generate_demo_data() -> Timetable:
    timeslots = [
        Timeslot(day, start, start.replace(hour=start.hour + 1))
        for day in ('MONDAY', 'TUESDAY')
        for start in (time(8, 30), time(9, 30), time(10, 30), time(13, 30), time(14, 30))
    ]

    rooms = [Room(f'Room {name}') for name in ('A', 'B', 'C')]

    lessons = []

    def id_generator():
        current = 0
        while True:
            yield str(current)
            current += 1
    ids = id_generator()
    lessons.append(Lesson(next(ids), "Math", "A. Turing", "9th grade"))
    lessons.append(Lesson(next(ids), "Math", "A. Turing", "9th grade"))
    lessons.append(Lesson(next(ids), "Physics", "M. Curie", "9th grade"))
    lessons.append(Lesson(next(ids), "Chemistry", "M. Curie", "9th grade"))
    lessons.append(Lesson(next(ids), "Biology", "C. Darwin", "9th grade"))
    lessons.append(Lesson(next(ids), "History", "I. Jones", "9th grade"))
    lessons.append(Lesson(next(ids), "English", "I. Jones", "9th grade"))
    lessons.append(Lesson(next(ids), "English", "I. Jones", "9th grade"))
    lessons.append(Lesson(next(ids), "Spanish", "P. Cruz", "9th grade"))
    lessons.append(Lesson(next(ids), "Spanish", "P. Cruz", "9th grade"))
    lessons.append(Lesson(next(ids), "Math", "A. Turing", "10th grade"))
    lessons.append(Lesson(next(ids), "Math", "A. Turing", "10th grade"))
    lessons.append(Lesson(next(ids), "Math", "A. Turing", "10th grade"))
    lessons.append(Lesson(next(ids), "Physics", "M. Curie", "10th grade"))
    lessons.append(Lesson(next(ids), "Chemistry", "M. Curie", "10th grade"))
    lessons.append(Lesson(next(ids), "French", "M. Curie", "10th grade"))
    lessons.append(Lesson(next(ids), "Geography", "C. Darwin", "10th grade"))
    lessons.append(Lesson(next(ids), "History", "I. Jones", "10th grade"))
    lessons.append(Lesson(next(ids), "English", "P. Cruz", "10th grade"))
    lessons.append(Lesson(next(ids), "Spanish", "P. Cruz", "10th grade"))

    return Timetable(timeslots, rooms, lessons)

## Solve it

Configure and run the solver:

In [8]:
from timefold.solver.config import (SolverConfig, ScoreDirectorFactoryConfig,
                                    TerminationConfig, Duration)
from timefold.solver import SolverFactory

solver_factory = SolverFactory.create(
    SolverConfig(
        solution_class=Timetable,
        entity_class_list=[Lesson],
        score_director_factory_config=ScoreDirectorFactoryConfig(
            constraint_provider_function=define_constraints
        ),
        termination_config=TerminationConfig(
            # The solver runs only for 5 seconds on this small dataset.
            # It's recommended to run for at least 5 minutes ("5m") otherwise.
            spent_limit=Duration(seconds=5)
        )
    ))

problem = generate_demo_data()

print(f'Solving the problem ...')
solver = solver_factory.build_solver()
solution = solver.solve(problem)
print(f'Solving finished with score ({solution.score}).')

Solving the problem ...
Solving finished with score (0hard/10soft).


## Visualize the schedule

Show the timetable:

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

### Lessons per teacher per weekday

### Lessons per teacher per hour

## Analyze the score

Let's break down the score per constraint:

And visualize it:

In [9]:
from timefold.solver import SolutionManager

solution_manager = SolutionManager.create(solver_factory)
score_analysis = solution_manager.analyze(solution)

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).